## **Get data and Libraries**

In [1]:
!git clone https://github.com/Vicomtech/hate-speech-dataset.git

Cloning into 'hate-speech-dataset'...
remote: Enumerating objects: 10785, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 10785 (delta 6), reused 0 (delta 0), pack-reused 10771
Receiving objects: 100% (10785/10785), 1.16 MiB | 8.33 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
!pip install transformers
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from transformers import BertForSequenceClassification, BertTokenizer
from transformers import get_linear_schedule_with_warmup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
folder_path = "hate-speech-dataset/all_files"
csv_path = "hate-speech-dataset/annotations_metadata.csv"

model_name = "bert-base-uncased"
device = "cuda:0"

tokenizer = BertTokenizer.from_pretrained(model_name)

# **Read and Preprocess Data**

In [6]:
df = pd.read_csv(csv_path)
print(df.shape)

(10944, 5)


In [7]:
df.label.value_counts()

noHate      9507
hate        1196
relation     168
idk/skip      73
Name: label, dtype: int64

In [8]:
df = df.set_index("file_id")
df["text"] = ""
df = df[df["label"].isin(["hate", "noHate"])]

In [9]:
df.shape

(10703, 5)

In [10]:
file_ids = df.index.tolist()

for index, row in df.iterrows():
  file_path = os.path.join(folder_path, f'{index}.txt')

  with open(file_path, 'r') as f:
    text = f.read()
  
  df.at[index, "text"] = text

In [11]:
df.head()

,user_id,subforum_id,num_contexts,label,text
file_id,,,,,
12834217_1,572066,1346,0,noHate,"As of March 13th , 2014 , the booklet had been..."
12834217_2,572066,1346,0,noHate,In order to help increase the booklets downloa...
12834217_3,572066,1346,0,noHate,( Simply copy and paste the following text int...
12834217_4,572066,1346,0,hate,Click below for a FREE download of a colorfull...
12834217_5,572066,1346,0,noHate,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...


In [12]:
df = df.reset_index(drop=True).drop(["user_id", "subforum_id", "num_contexts"], axis=1)

In [13]:
stopw = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
  text = text.lower()

  text = " ".join([w for w in text.split() if w not in stopw])

  pattern = re.compile(r"\w+")
  text = " ".join(re.findall(pattern, text))
  text = " ".join([lemmatizer.lemmatize(w) for w in text.split()])

  return text

In [14]:
df["clean_text"] = df["text"].apply(preprocess)

In [15]:
encoder = LabelEncoder()

df["sentiment"] = encoder.fit_transform(df["label"])

In [16]:
df.head()

,label,text,clean_text,sentiment
0,noHate,"As of March 13th , 2014 , the booklet had been...",march 13th 2014 booklet downloaded 18 300 time...,1
1,noHate,In order to help increase the booklets downloa...,order help increase booklet downloads would gr...,1
2,noHate,( Simply copy and paste the following text int...,simply copy paste following text youtube video...,1
3,hate,Click below for a FREE download of a colorfull...,click free download colorfully illustrated 132...,0
4,noHate,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,click download 7 42 mb green banner link,1


In [17]:
from sklearn.model_selection import train_test_split

df = df.sample(5000, random_state=42)

train_data, test_data = train_test_split(df, test_size=0.15, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.20, random_state=42)

In [18]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

In [19]:
train_data.shape, test_data.shape, val_data.shape

((3400, 4), (750, 4), (850, 4))

In [20]:
train_labels = torch.tensor(train_data['sentiment'])
train_encodings = tokenizer(
                        train_data["clean_text"].tolist(), 
                        max_length=512, 
                        add_special_tokens=True,
                        truncation=True,
                        padding="max_length", 
                        return_tensors="pt"
                    )

val_labels = torch.tensor(val_data['sentiment'], dtype=torch.float32)
val_encodings = tokenizer(
                        val_data["clean_text"].tolist(), 
                        max_length=512, 
                        add_special_tokens=True,
                        truncation=True,
                        padding="max_length", 
                        return_tensors="pt"
                    )

test_labels = torch.tensor(test_data['sentiment'], dtype=torch.float32)
test_encodings = tokenizer(
                        test_data["clean_text"].tolist(), 
                        max_length=512, 
                        add_special_tokens=True,
                        truncation=True,
                        padding="max_length", 
                        return_tensors="pt"
                    )

In [21]:
batch_size = 8

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)

train_dl = DataLoader(train_dataset, batch_size=batch_size, 
                      shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size)
val_dl = DataLoader(val_dataset, batch_size=batch_size)

Oversampling

In [22]:
from imblearn.over_sampling import RandomOverSampler

In [23]:
oversampler = RandomOverSampler(random_state=42)

In [24]:
input_ids, labels = oversampler.fit_resample(train_dataset.tensors[0],  
                         train_dataset.tensors[2])

In [25]:
attention_masks = []

for input_id in input_ids:
  mask = [1 if token != 0 else 0 for token in input_id]
  attention_masks.append(mask)

In [26]:
input_ids = torch.tensor(input_ids, dtype=torch.long)
attention_mask = torch.tensor(attention_masks, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.float32)

In [27]:
train_dataset = TensorDataset(input_ids, attention_mask, labels)

In [28]:
batch_size = 8

train_dl = DataLoader(train_dataset, batch_size=batch_size, 
                      shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size)
val_dl = DataLoader(val_dataset, batch_size=batch_size)

# **Build and Train Model**

In [29]:
class HateSpeechDetector(nn.Module):
  def __init__(self):
    super(HateSpeechDetector, self).__init__()
    self.model = BertForSequenceClassification.from_pretrained(model_name)
  
  def forward(self, input_ids, attention_mask=None):
    bert_output = self.model(input_ids=input_ids, 
                           attention_mask=attention_mask, 
                           return_dict=False)
    return bert_output[0]

In [ ]:
hsModel = HateSpeechDetector()
hsModel.to(device)

In [31]:
from tqdm import tqdm

In [32]:
def compute_accuracy(logits, labels):
  predicted_labels = torch.argmax(logits, axis=-1)
  labels = torch.argmax(labels, axis=-1)

  correct_labels = (predicted_labels == labels).sum().item()
  accuracy = correct_labels/labels.size(0)

  return round(accuracy, 3)

In [33]:
def printEpochResult(epoch, train_loss, val_loss, train_acc, val_acc):
  print(f"\nEpoch: {epoch+1}/{epochs}")
  print(f"Train Accuracy: {train_acc:.4f}\tVal. Accuracy: {val_acc:4f}")
  print(f"Train Loss: {train_loss:.4f}\tVal. Loss: {val_loss:.4f}\n")

In [35]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss

epochs = 5
total_steps = len(train_dl) * epochs
criterion = BCEWithLogitsLoss()
optimizer = Adam(hsModel.parameters(), lr=2e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=total_steps)
checkpoint = "best_model.pth"

best_accuracy = 0

for epoch in range(epochs):
  train_acc = 0
  val_acc = 0
  train_loss = 0
  val_loss = 0
  hsModel.train()
  for batch in tqdm(train_dl, total=len(train_dl)):
    input_ids = batch[0].squeeze().to(device)
    attention_mask = batch[1].to(device)
    labels = torch.eye(2)[torch.tensor(batch[2], dtype=torch.long)].to(device)  # shape:[8] -> shape:[8, 2]

    optimizer.zero_grad()
    outputs = hsModel(input_ids=input_ids, 
                    attention_mask=attention_mask)

    loss = criterion(outputs, labels)

    train_loss += loss.item()
    train_acc += compute_accuracy(outputs, labels)

    loss.backward()
    optimizer.step()
    scheduler.step()
  
  hsModel.eval()
  with torch.no_grad():
    for batch in tqdm(val_dl, total=len(val_dl)):
      input_ids = batch[0].squeeze().to(device)
      attention_mask = batch[1].to(device)
      labels = torch.eye(2)[torch.tensor(batch[2], dtype=torch.long)].to(device) 
    
      outputs = hsModel(input_ids=input_ids, 
                      attention_mask=attention_mask)

      loss = criterion(outputs, labels)

      val_loss += loss.item()
      val_acc += compute_accuracy(outputs, labels)

  # Calculate average accuracy of entire epoch
  total_train_accuracy = round(train_acc / len(train_dl), 4)
  total_val_accuracy = round(val_acc / len(val_dl), 4)

  # Save best model
  if total_val_accuracy > best_accuracy:
    best_accuracy = total_val_accuracy
    print("\n->Saving Best Model!.......")
    torch.save(hsModel.state_dict(), checkpoint)

  printEpochResult(epoch, train_loss/len(train_dl), val_loss/len(val_dl), total_train_accuracy, total_val_accuracy)

100%|██████████| 107/107 [00:27<00:00,  3.83it/s]



->Saving Best Model!.......

Epoch: 1/5
Train Accuracy: 0.8765	Val. Accuracy: 0.908900
Train Loss: 0.2867	Val. Loss: 0.3007



100%|██████████| 107/107 [00:27<00:00,  3.84it/s]



Epoch: 2/5
Train Accuracy: 0.9836	Val. Accuracy: 0.894900
Train Loss: 0.0502	Val. Loss: 0.4147



100%|██████████| 107/107 [00:27<00:00,  3.84it/s]



->Saving Best Model!.......

Epoch: 3/5
Train Accuracy: 0.9967	Val. Accuracy: 0.910000
Train Loss: 0.0099	Val. Loss: 0.4376



100%|██████████| 107/107 [00:27<00:00,  3.84it/s]



->Saving Best Model!.......

Epoch: 4/5
Train Accuracy: 0.9998	Val. Accuracy: 0.911200
Train Loss: 0.0024	Val. Loss: 0.4848



100%|██████████| 107/107 [00:27<00:00,  3.83it/s]



->Saving Best Model!.......

Epoch: 5/5
Train Accuracy: 1.0000	Val. Accuracy: 0.912400
Train Loss: 0.0013	Val. Loss: 0.4862



## **Inference**

In [37]:
# Load Model
inference_model = HateSpeechDetector()

inference_model.load_state_dict(torch.load("best_model.pth"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [38]:
def predict_speech(speech):
  speech = preprocess(speech)
  tokenized_speech = tokenizer(
                        speech, 
                        max_length=512, 
                        add_special_tokens=True,
                        truncation=True,
                        padding="max_length", 
                        return_tensors="pt"
                    )
  
  input_ids = tokenized_speech["input_ids"]
  attention_mask = tokenized_speech["attention_mask"]
  
  logits = inference_model(input_ids, attention_mask)

  output = torch.argmax(logits, axis=-1)

  if int(output) == 1:
    print("Fair Speech")
  else:
    print("Hate Speech")

In [58]:
sentence_1 = """Yep the bottom line is without a strong family behind her a young girl is very vulnerable .. 
              The Jew knows this and Glamorises the Negro using its media power , here we see the results ."""

sentence_2 = "I think that white women who cannot find a suitable white partner for whatever reason should not delay childbearing but should go to a \
            sperm bank and start having white kids ."

sentence_3 = "Are you an anti-Semite who is trying to undermine the racial integrity of the world 's only Jewish state ?"

In [46]:
predict_speech(sentence_1)

Hate Speech


In [55]:
predict_speech(sentence_2)

Fair Speech


In [59]:
predict_speech(sentence_3)

Hate Speech
